はじめに
-------

Hyにより Chainerのサンプルを記述します。

Chainerのインストール
-----------------

In [ ]:
!pip install chainer -U

Hyによる Chainerチュートリアル
-----------------

Hyで記述した [Chainerのチュートリアル](https://docs.chainer.org/en/stable/tutorial/basic.html#core-concept)のコードを以下に示します。

```python
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
```

In [ ]:
(import [numpy :as np])
(import chainer)
(import [chainer [cuda Function gradient_check report training utils Variable
                  datasets iterators optimizers serializers Link Chain ChainList]])
(import [chainer.functions :as F]
        [chainer.links :as L])
(import [chainer.training [extensions]])

```python
x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)

y = x**2 - 2 * x + 1
y.data
```

In [ ]:
(def x_data (np.array 5 :dtype np.float32))
(def x (Variable x_data))
(def y (+ (** x 2) (* -2 x) 1))  ;;y = x**2 - 2 * x + 1
(print y.data)

```python
y.backward()
x.grad
```

In [ ]:
(y.backward)
(print x.grad)

```python
z = 2*x
y = x**2 - z + 1
y.backward(retain_grad=True)
z.grad
```

In [ ]:
(def z (* 2 x))
(def y (+ (** 2 x) (- z) 1))
(.backward y :retain_grad True)
(print z.grad)

```python
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = x**2 - 2*x + 1
y.grad = np.ones((2, 3), dtype=np.float32)
y.backward()
x.grad
```

In [ ]:
(def x (Variable (np.array [[1 2 3] [4 5 6]] :dtype np.float32)))
(def y (+ (** 2 x) (* -2 x) 1))
(def y.grad (np.ones [2 3] :dtype np.float32))
(y.backward)
(print x.grad)

```python
f = L.Linear(3, 2)
f.W.data
f.b.data
```

In [ ]:
(def f (L.Linear 3 2))
(print f.W.data)
(print f.b.data)

```python
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = f(x)
y.data
```

In [ ]:
(def x (Variable (np.array [[1 2 3] [4 5 6]] :dtype np.float32)))
(def y (f x))
(print y.data)

```python
f.cleargrads()
y.grad = np.ones((2, 2), dtype=np.float32)
y.backward()
f.W.grad
f.b.grad
```

In [ ]:
(f.cleargrads)
(def y.grad (np.ones [2 2] :dtype np.float32))
(y.backward)
(print f.W.grad)
(print f.b.grad)

```python
l1 = L.Linear(4, 3)
l2 = L.Linear(3, 2)
def my_forward(x):
   h = l1(x)
   return l2(h)
```

In [ ]:
(def l1 (L.Linear 4 3))
(def l2 (L.Linear 3 2))
(defn my_forward [x]
  (def h (l1 x))
  (l2 h))

```python
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4, 3)
        self.l2 = L.Linear(3, 2)

    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)
```

In [ ]:
(defclass MyProc [object]
  (defn --init-- [self]
    (def self.l1 (L.Linear 4 3))
    (def self.l2 (L.Linear 3 2)))
  (defn forward [self x]
    (def h (self.l1 x))
    (self.l2 h)))

```python
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(4, 3)
            self.l2 = L.Linear(3, 2)

    def __call__(self, x):
        h = self.l1(x)
        return self.l2(h)
```

In [ ]:
(defclass MyChain [Chain]
  (defn --init-- [self]
    (.__init__ (super MyChain self))
    (with [(self.init_scope)]
      (def self.l1 (L.Linear 4 3))
      (def self.l2 (L.Linear 3 2))))

  (defn --call-- [self x]
    (def h (self.l1 x))
    (self.l2 h)))

```python
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4, 3),
            L.Linear(3, 2),
        )

    def __call__(self, x):
        h = self[0](x)
        return self[1](h)
```

In [ ]:
(defclass MyChain2 [ChainList]
  (defn --init-- [self]
    (.__init__ (super MyChain2 self)
                 (L.Linear 4 3)
                 (L.Linear 3 2)))

  (defn --call-- [self x]
    (def h (self[0] x))
    (self[1] h)))

```python
model = MyChain()
optimizer = optimizers.SGD()
optimizer.setup(model)
```

In [ ]:
(def model (MyChain))
(def optimizer (optimizers.SGD))
(optimizer.setup model)

```python
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))
```

In [ ]:
(optimizer.add_hook (chainer.optimizer.WeightDecay 0.0005))

```python
x = np.random.uniform(-1, 1, (2, 4)).astype('f')
model.cleargrads()
# compute gradient here...
loss = F.sum(model(chainer.Variable(x)))
loss.backward()
optimizer.update()
```

In [ ]:
(def x (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(model.cleargrads)
;; compute gradient here...
(def loss (F.sum (model (chainer.Variable x))))
(loss.backward)
(optimizer.update)

```python
def lossfun(arg1, arg2):
    # calculate loss
    loss = F.sum(model(arg1 - arg2))
    return loss
arg1 = np.random.uniform(-1, 1, (2, 4)).astype('f')
arg2 = np.random.uniform(-1, 1, (2, 4)).astype('f')
optimizer.update(lossfun, chainer.Variable(arg1), chainer.Variable(arg2))
```

In [ ]:
(defn lossfun [arg1 arg2]
  (def loss (F.sum (model (- arg1 arg2))))
  loss)

(def arg1 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(def arg2 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(optimizer.update lossfun (chainer.Variable arg1) (chainer.Variable arg2))

```python
serializers.save_npz('my.model', model)
```

In [ ]:
(serializers.save_npz "my.model" model)

```python
train, test = datasets.get_mnist()
```

In [ ]:
(def [train test] (datasets.get_mnist))

```python
train_iter = iterators.SerialIterator(train, batch_size=100, shuffle=True)
```

In [ ]:
(def train_iter (iterators.SerialIterator train :batch_size 100 :shuffle True))

```python
class MLP(Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y
```

In [ ]:
(defclass MLP [Chain]
  (defn --init-- [self n_units n_out]
    (.__init__ (super (MLP self)))
    (with [(self.init_scope)]
      (def self.l1 (L.Linear None n_units))  ;; n_in -> n_units
      (def self.l2 (L.Linear None n_units))  ;; n_units -> n_units
      (def self.l3 (L.Linear None n_out))))  ;; n_units -> n_out

  (defn --call-- [self, x]
    (def h1 (F.relu (self.l1 x)))
    (def h2 (F.relu (self.l2 h1)))
    (def y (self.l3 h2))
    y))

```python
model = L.Classifier(MLP(100, 10))  # the input size, 784, is inferred
optimizer = optimizers.SGD()
optimizer.setup(model)
```

In [ ]:
(def model (L.Classifier MLP 100 10))  ;; the input size, 784, is inferred
(def optimizer (optimizers.SGD))
(optimizer.setup model)

```python
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out='result')
```

In [ ]:
(def updater (training.StandardUpdater train_iter optimizer))
(def trainer (training.Trainer updater [20 "epoch"] :out "result"))

```python
trainer.extend(extensions.Evaluator(train_iter, model))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.ProgressBar())
trainer.run()
```

In [ ]:
(trainer.extend (extensions.Evaluator train_iter model))
(trainer.extend (extensions.LogReport))
(trainer.extend (extensions.PrintReport ["epoch" "main/accuracy" "validation/main/accuracy"]))
(trainer.extend (extensions.ProgressBar))
(trainer.run)

In [ ]:
!ls -lRF